In [13]:
import numpy as np
import random

# Define the grid (0: open cell, 1: wall)
grid = [
    [0, 0, 1, 0, 0],
    [0, 1, 0, 1, 0],
    [0, 0, 0, 0, 0],
    [1, 1, 1, 1, 0],
    [0, 0, 0, 1, 0]
]

# Hyperparameters
learning_rate = 0.1
discount_factor = 0.9
exploration_rate = 1.0
exploration_decay = 0.99
min_exploration_rate = 0.1
num_episodes = 1000

# Initialize Q-table
n_states = len(grid) * len(grid[0])
n_actions = 4  # Up, Down, Left, Right
q_table = np.zeros((n_states, n_actions))

def state_to_index(row, col):
    return row * len(grid[0]) + col

def index_to_state(index):
    return divmod(index, len(grid[0]))

def get_available_actions(state):
    row, col = index_to_state(state)
    actions = []
    if row > 0 and grid[row - 1][col] == 0:  # Up
        actions.append(0)
    if row < len(grid) - 1 and grid[row + 1][col] == 0:  # Down
        actions.append(1)
    if col > 0 and grid[row][col - 1] == 0:  # Left
        actions.append(2)
    if col < len(grid[0]) - 1 and grid[row][col + 1] == 0:  # Right
        actions.append(3)
    return actions

def train_agent():
    global exploration_rate
    for episode in range(num_episodes):
        state = state_to_index(0, 0)  # Start position
        done = False

        while not done:
            if random.uniform(0, 1) < exploration_rate:
                action = random.choice(get_available_actions(state))  # Explore
            else:
                action = np.argmax(q_table[state])  # Exploit

            # Determine new position based on action
            new_row, new_col = index_to_state(state)
            if action == 0:  # Up
                new_row -= 1
            elif action == 1:  # Down
                new_row += 1
            elif action == 2:  # Left
                new_col -= 1
            elif action == 3:  # Right
                new_col += 1

            # Check if the new position is valid
            if 0 <= new_row < len(grid) and 0 <= new_col < len(grid[0]) and grid[new_row][new_col] == 0:
                new_state = state_to_index(new_row, new_col)
            else:
                new_state = state  # If invalid, stay in the same state

            reward = -1  # Negative cost for each move
            if (new_row, new_col) == (len(grid) - 1, len(grid[0]) - 1):
                reward = 10  # Reward for reaching the goal
                done = True  # End episode

            # Update Q-value
            best_future_q = np.max(q_table[new_state])
            q_table[state][action] += learning_rate * (reward + discount_factor * best_future_q - q_table[state][action])

            state = new_state

        # Decay exploration rate
        if exploration_rate > min_exploration_rate:
            exploration_rate *= exploration_decay

train_agent()

# Display Q-table
print("Q-table:")
print(q_table)

# Test the learned policy
def test_agent():
    state = state_to_index(0, 0)
    path = []

    while state != state_to_index(len(grid) - 1, len(grid[0]) - 1):
        path.append(state)
        action = np.argmax(q_table[state])
        new_row, new_col = index_to_state(state)

        if action == 0:  # Up
            new_row -= 1
        elif action == 1:  # Down
            new_row += 1
        elif action == 2:  # Left
            new_col -= 1
        elif action == 3:  # Right
            new_col += 1

        # Check if the new position is valid
        if 0 <= new_row < len(grid) and 0 <= new_col < len(grid[0]) and grid[new_row][new_col] == 0:
            state = state_to_index(new_row, new_col)

    path.append(state)  # Add the goal state
    return path

# Get the path from start to finish
path = test_agent()
print("Learned path:", path)


Q-table:
[[-2.37657286 -0.434062   -2.37657286 -2.2637506 ]
 [-2.14321859 -2.14321859 -1.39258132 -2.14321859]
 [ 0.          0.          0.          0.        ]
 [-0.4900995  -0.3940399  -0.3940399  -0.39862095]
 [-0.3940399   2.64371958 -1.12375735 -0.3940399 ]
 [-1.3943233   0.62882    -1.9836941  -1.90272132]
 [ 0.          0.          0.          0.        ]
 [-0.199       3.11861116 -0.199      -0.199     ]
 [ 0.          0.          0.          0.        ]
 [-0.10174936  6.19906794 -0.1         0.        ]
 [-0.43797431 -1.31254187 -1.22478977  1.8098    ]
 [-0.58519851 -0.58519851  0.62813428  3.122     ]
 [ 1.78654491 -0.67934652  1.80381424  4.58      ]
 [-0.1         0.          3.12108081  6.2       ]
 [ 4.57197077  8.          4.57889704  0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 6.19977513 10.     